In [ ]:
from flask import Flask, render_template, redirect, url_for, request, flash
from werkzeug.security import generate_password_hash, check_password_hash
import pandas as pd
from flask_sqlalchemy import SQLAlchemy
from flask_login import UserMixin, login_user, login_required, logout_user, current_user, LoginManager
from flask_wtf import FlaskForm
from flask_bcrypt import Bcrypt
from flask_admin import Admin
from flask_admin.contrib.sqla import ModelView
from datetime import datetime

app = Flask(__name__)
app.config['FLASK_ADMIN_SWATCH'] = 'cerulean'
admin = Admin(app, name='管理员', template_mode='bootstrap3')
app.config['SECRET_KEY'] = 'thisissecretkey'

db = SQLAlchemy(app)#create the database instance
bcrypt = Bcrypt(app)
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///demoapp.db'#connect app files to database? Not sure

db = SQLAlchemy(app)
login_manager = LoginManager()
login_manager.init_app(app)

class User(db.Model):
    __tablename__ = 'users'
    id = db.Column(db.Integer, primary_key=True, autoincrement=True)
    username = db.Column(db.String(50),unique=True)
    email = db.Column(db.String(50),unique=True)
    password = db.Column(db.String(20))
    is_admin=db.Column(db.Integer)
    
    def __repr__(self):
        return '<User %r>' % self.id

admin.add_view(ModelView(User, db.session))

class Todo(db.Model):
    id = db.Column(db.Integer, primary_key=True, autoincrement=True)
    content = db.Column(db.String(200), nullable=False)
    date_expected = db.Column(db.DateTime, nullable=False)

    def __repr__(self):
        return '<Task %r>' % self.id
    
admin.add_view(ModelView(Todo, db.session))
    
@login_manager.user_loader
def load_user(user_id):
    return User.query.get(int(user_id))

# WEB ROUTES FOR CONTROLLING ACCESS TO TEMPLATE VIEWS

@app.route("/")
def index():
    return render_template('index.html')

@app.route("/admin")
def admin():
    return render_template("admin/index.html")

@app.route("/blogs")
def blogs():
    df = pd.read_csv("shopping_info.csv")
    list = df.to_dict('records')
    return render_template('blogs.html',entries = list)

@app.route("/about")
def about():
    return render_template('about.html')

@app.route("/post")
def post():
    return render_template('post.html')

@app.route("/contact")
def contact():
    return render_template('contact.html')

@app.route("/dashboard")
def dashboard():
    return render_template('dashboard.html')

@app.route("/register", methods=["GET", "POST"])#这里只注册
def register():
    if request.method == "POST":
        username = request.form['username']#后面是页面输入框表单名字  前面是变量
        email = request.form['email']
        password = request.form['password']
        print(username,email,password)
        new_user = User(username=username,email=email,password=password,is_admin=0) #利用临时变量的值传入__init.py   
        try:
            db.session.add(new_user)
            db.session.commit()#上传
            print("Registered!")
            return redirect(url_for('register'))
        except Exception as e:
            print("Exception occurred.",e)
            return 'Fail to add user.'
    else:
        return render_template('register.html')

@app.route('/login', methods=["GET", "POST"])  # Login Here!
def login():
    if request.method == "POST":
        # Retrieve information
        inputted_email = request.form['email']
        inputted_password = request.form['password']

        # Send a query to db to retrieve user
        retrievedUser = db.session.query(User).filter_by(email=inputted_email).first()

        if not retrievedUser:  # User not found
            print("This User does not exist")
            flash("Wrong username or username not found.")
            return render_template('login.html')

        correct_password = generate_password_hash(retrievedUser.password)
        if check_password_hash(correct_password, inputted_password):  # Password correct
            print("Login Success!")
            flash("Successfully logged in.")
            if retrievedUser.is_admin==1:
                return redirect(url_for('admin'))
            return redirect(url_for('index'))
        else:  # Password incorrect
            print("Incorrect password.")
            flash("Wrong password.")
            return render_template('login.html')

    else:
        return render_template('login.html')
    
@app.route("/logout")
def logout():
    print('logout user.')
    flash('logged out.', current_user.username)
    logout_user()
    redirect(url_for('index'))
    return redirect(url_for('index'))   



    
#########################################################################################

if __name__ == "__main__":
    app.run('localhost', 2021)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


/Users/sophiawang/opt/anaconda3/lib/python3.8/site-packages/flask_sqlalchemy/__init__.py:851: UserWarning: Neither SQLALCHEMY_DATABASE_URI nor SQLALCHEMY_BINDS is set. Defaulting SQLALCHEMY_DATABASE_URI to "sqlite:///:memory:".
  warnings.warn(
/Users/sophiawang/opt/anaconda3/lib/python3.8/site-packages/flask_sqlalchemy/__init__.py:872: FSADeprecationWarning: SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and will be disabled by default in the future.  Set it to True or False to suppress this warning.
  warnings.warn(FSADeprecationWarning(
/Users/sophiawang/opt/anaconda3/lib/python3.8/site-packages/flask_sqlalchemy/__init__.py:872: FSADeprecationWarning: SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and will be disabled by default in the future.  Set it to True or False to suppress this warning.
  warnings.warn(FSADeprecationWarning(
 * Running on http://localhost:2021/ (Press CTRL+C to quit)
127.0.0.1 - - [29/Oct/2021 23:39:51] "GET / HTTP/1.1" 200 -
127.0.0.1 - -

Login Success!


127.0.0.1 - - [29/Oct/2021 23:40:01] "GET /admin/static/vendor/select2/select2.min.js?v=3.5.2 HTTP/1.1" 200 -
127.0.0.1 - - [29/Oct/2021 23:40:01] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [29/Oct/2021 23:40:03] "GET /admin/user/ HTTP/1.1" 200 -
127.0.0.1 - - [29/Oct/2021 23:40:03] "GET /admin/static/vendor/select2/select2.css?v=3.5.2 HTTP/1.1" 200 -
127.0.0.1 - - [29/Oct/2021 23:40:03] "GET /admin/static/vendor/bootstrap-daterangepicker/daterangepicker-bs3.css?v=1.3.22 HTTP/1.1" 200 -
127.0.0.1 - - [29/Oct/2021 23:40:03] "GET /admin/static/vendor/select2/select2-bootstrap3.css?v=1.4.6 HTTP/1.1" 200 -
127.0.0.1 - - [29/Oct/2021 23:40:03] "GET /admin/static/admin/js/filters.js?v=1.0.0 HTTP/1.1" 200 -
127.0.0.1 - - [29/Oct/2021 23:40:03] "GET /admin/static/vendor/bootstrap-daterangepicker/daterangepicker.js?v=1.3.22 HTTP/1.1" 200 -
127.0.0.1 - - [29/Oct/2021 23:40:03] "GET /admin/static/admin/js/form.js?v=1.0.1 HTTP/1.1" 200 -
127.0.0.1 - - [29/Oct/2021 23:40:03] "GET /admin/static

In [1]:
import sqlite3 
con = sqlite3.connect('demoapp.db')
cursor = con.cursor()
sql = "CREATE TABLE Todo (id INTEGER, content CHAR(200), date_expected DATE);"
try:
    cursor.execute(sql)
    con.commit()
    print("Table successfully created.")
except Exception as e:
    print("Exception occurred.",e)
cursor.close()
con.close()

Table successfully created.


In [ ]:
from flask import Flask, render_template, redirect, url_for, request, flash
from werkzeug.security import generate_password_hash, check_password_hash
import pandas as pd
from flask_sqlalchemy import SQLAlchemy
from flask_login import UserMixin, login_user, login_required, logout_user, current_user, LoginManager
from flask_wtf import FlaskForm
from flask_bcrypt import Bcrypt
from flask_admin import Admin
from flask_admin.contrib.sqla import ModelView
from datetime import datetime

app = Flask(__name__)
app.config['FLASK_ADMIN_SWATCH'] = 'cerulean'
admin = Admin(app, name='管理员', template_mode='bootstrap3')

db = SQLAlchemy(app)#create the database instance
bcrypt = Bcrypt(app)
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///demoapp.db'#connect app files to database? Not sure
app.config['SECRET_KEY'] = 'secretkey'

db = SQLAlchemy(app)
login_manager = LoginManager()
login_manager.init_app(app)
login_manager.login_message = "Please login."
login_manager.refresh_view = 'login'
login_manager.login_view = 'login'

class User(db.Model):
    __tablename__ = 'users'
    id = db.Column(db.Integer, primary_key=True, autoincrement=True)
    username = db.Column(db.String(50),unique=True)
    email = db.Column(db.String(50),unique=True)
    password = db.Column(db.String(20))
    is_admin=db.Column(db.Integer)
    
    def __repr__(self):
        return '<User %r>' % self.id

admin.add_view(ModelView(User, db.session))

class Todo(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    content = db.Column(db.String(200), nullable=False)
    date_created = db.Column(db.DateTime, default=datetime.utcnow)

    def __repr__(self):
        return '<Task %r>' % self.id

@login_manager.user_loader
def load_user(user_id):
    return User.query.get(int(user_id))

# WEB ROUTES FOR CONTROLLING ACCESS TO TEMPLATE VIEWS

@app.route("/")
def index():
    return render_template('index.html')

@app.route("/admin")
def admin():
    return render_template("admin/index.html")

@app.route("/blogs")
def blogs():
    df = pd.read_csv("shopping_info.csv")
    list = df.to_dict('records')
    return render_template('blogs.html',entries = list)

@app.route("/about")
def about():
    return render_template('about.html')

@app.route("/post")
def post():
    return render_template('post.html')

@app.route("/contact")
def contact():
    return render_template('contact.html')

@app.route("/dashboard")
def dashboard():
    return render_template('dashboard.html')

@app.route("/register", methods=["GET", "POST"])#这里只注册
def register():
    if request.method == "POST":
        username = request.form['username']#后面是页面输入框表单名字  前面是变量
        email = request.form['email']
        password = request.form['password']
        print(username,email,password)
        new_user = User(username=username,email=email,password=password,is_admin=0) #利用临时变量的值传入__init.py   
        try:
            db.session.add(new_user)
            db.session.commit()#上传
            print("Registered!")
            return redirect(url_for('register'))
        except Exception as e:
            print("Exception occurred.",e)
            return 'Fail to add user.'
    else:
        return render_template('register.html')

@app.route('/login', methods=["GET", "POST"])  # Login Here!
def login():
    if request.method == "POST":
        # Retrieve information
        inputted_email = request.form['email']
        inputted_password = request.form['password']

        # Send a query to db to retrieve user
        retrievedUser = db.session.query(User).filter_by(email=inputted_email).first()

        if not retrievedUser:  # User not found
            print("This User does not exist")
            flash("Wrong username or username not found.")
            return render_template('login.html')

        correct_password = generate_password_hash(retrievedUser.password)
        if check_password_hash(correct_password, inputted_password):  # Password correct
            print("Login Success!")
            flash("Successfully logged in.")
            if retrievedUser.is_admin==1:
                return redirect(url_for('admin'))
            return redirect(url_for('index'))
        else:  # Password incorrect
            print("Incorrect password.")
            flash("Wrong password.")
            return render_template('login.html')

    else:
        return render_template('login.html')
    
@app.route("/logout")
@login_required
def logout():
    return render_template('logout.html')    


@app.route("/todo")
@login_required
def todo():
    print('logout user.')
    flash('logged out.', current_user.username)
    logout_user()
    redirect(url_for('index')) 

if __name__ == "__main__":
    app.run('localhost', 2021)

In [ ]:
@app.route("/todo",methods=['POST', 'GET'])
def todo():
    if request.method == 'POST':
        task_content = request.form['content']
        new_task = Todo(content=task_content)

        try:
            db.session.add(new_task)
            db.session.commit()
            return redirect('/todo')
        except:
            return 'There was an issue adding your task'

    else:
        tasks = Todo.query.order_by(Todo.date_created).all()
        return render_template('todo.html', tasks=tasks)

@app.route('/delete/<int:id2>')
def delete(id2):
    task_to_delete = Todo.query.get_or_404(id2)

    try:
        db.session.delete(task_to_delete)
        db.session.commit()
        return redirect('/todo')
    except:
        return 'There was a problem deleting that task'

@app.route('/update/<int:id2>', methods=['GET', 'POST'])
def update(id2):
    task = Todo.query.get_or_404(id2)

    if request.method == 'POST':
        task.content = request.form['content']

        try:
            db.session.commit()
            return redirect('/todo')
        except:
            return 'There was an issue updating your task'

    else:
        return render_template('update.html', task=task)
    
